# Operaciones básicas con ElasticSearch
El objetivo de esta práctica es aprender a utilizar los comandos básicos de ElasticSearch. Indexar, borrar, editar y buscar documentos almacenados en este motor de búsqueda.

## Introducción

Este primer bloque de código sirve para configurar el Notebook

In [1]:
from IPython.display import JSON

Ahora vamos a descargar el cliente de ElasticSearch en Python.

In [2]:
pip install elasticsearch==7.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 1.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


Por último, creamos la conexion con el servidor de Elastic Search desplegado

In [3]:
from elasticsearch import Elasticsearch
es = Elasticsearch(
    ['elasticsearch']
)
JSON(es.info())

<IPython.core.display.JSON object>

## Ejercicio1. Monitorizando el estado de tu cluster.
La idea de este ejercicio es que conozcamos cómo extraer los principales parámetros de un cluster, para conocer su estado.

### Estado del cluster
El comando más básico para saber cual es el estado a alto de nivel del cluster.

In [4]:
JSON(es.cluster.stats())

<IPython.core.display.JSON object>

Si queremos una información más detallada podemos lanzar el comando `state`

In [5]:
JSON(es.cluster.state())

<IPython.core.display.JSON object>

Podemos filtra algunos campos si queremos información sobre algo específico.

In [6]:
JSON(es.cluster.state(metric=["metadata","routing_table"]))

<IPython.core.display.JSON object>

### Health check
Para poder revisar el estado del cluster, debemos usar el método `health`. Esta función nos devolverá información de cómo se encuentra el cluster. El método devuelve un codigo de color:
- **Verde**. El cluster funciona correctamente y todas los shards están disponibles.
- **Amarillo**. El cluster permite indexar y buscar información porque todos los primary shard estan desplegados.
- **Rojo**. El cluster esta inestable y algunos shards no funcionan correctamente.

In [7]:
JSON(es.cluster.health())

<IPython.core.display.JSON object>

El comando permite lanzar espera activas para determinados estados.

In [8]:
JSON(es.cluster.health(wait_for_status="yellow",timeout="5s"))

<IPython.core.display.JSON object>

### Settings
Otro comando que puede ayudarte a entender como se encuentra tu servidor es el comando `settings`, el cual devuelve la configuración del servidor elastic search al que estamos conectados.

In [101]:
JSON(es.cluster.get_settings(include_defaults="true"))

<IPython.core.display.JSON object>

## Ejercicio 2. Creacion de indices
En esta sección vamos a explicar la funciones básicas de un índice. Como crearlo y como indicar cual es su esquema por defecto.

El comando `create`crea un indice con configración básica.

In [17]:
JSON(es.indices.create(index="prueba"))

<IPython.core.display.JSON object>

Para ver la configuración de este índice podemos llamar al método `get`.

In [18]:
JSON(es.indices.get(index="prueba"))

<IPython.core.display.JSON object>

El comando create falla si el indice ya existe por tanto si queremos ejecutar el create de una forma segura debemos utilizar el comandos `exists`.

In [104]:
if es.indices.exists(index="prueba"):
    print("El indice ya existe")
else:
    JSON(es.indices.create(index="prueba"))
    

El indice ya existe


Para borrar un indice utilizamos el comando `delete`.

In [105]:
JSON(es.indices.delete(index="prueba"))

<IPython.core.display.JSON object>

Si queremos cambiar el numero de shards o replicas debemos utilizar el parametro `body` e introducir una estructura de `settings`.

In [106]:
request_body= {
    "settings": {
        "number_of_shards":1,
        "number_of_replicas":0
    }
}
JSON(es.indices.create(index="prueba", body=request_body))

<IPython.core.display.JSON object>

Para comprobar los cambios ejecutamos el comando `get`.

In [107]:
JSON(es.indices.get(index="prueba"))

<IPython.core.display.JSON object>

Cuando creamos un indice podemos indicar el esquema que vamos a usar. Por defecto elastic search infiere el esquema de los datos que vamos introduciendo, pero esta inferencia no es suficiente si queremos controlara la búsqueda. Para eso tenemos que modificar el `body` durante la creación añadiendo un elemento `mapping` o podemos actualizar el indice utilizando el método `put_mapping`y ñadiendo en parametro `body` la definición.

In [108]:
request_body= {
    "properties": {
        "email": {
            "type":"keyword"
        }
    }
}
JSON(es.indices.put_mapping(index="prueba", body=request_body))

<IPython.core.display.JSON object>

Comprobamos que el mapping ha sido actualizado con el comando `get_mapping`.

In [109]:
JSON(es.indices.get_mapping(index="prueba"))

<IPython.core.display.JSON object>

### Práctica
En esta sección vamos a repasar lo comando que hemos esta ejecutando en este ejercicio. Para ello vamos a desarrollar piezas de codigo que cumplan las indicaciones.

**Escribe el código que permita eliminar el indice prueba. El comando debe poderse ejecutar varias veces sin fallar.**

In [21]:
#TODO - Escribe aquí tu codígo 

El indice no existe


**Escribe el codigo que permita crear un indice llamado "example" con un shard y una replica y que tenga un esquema con un campo title de tipo keyword y un campo description de tipo text. Comprueba que el indice esta bien creado. El codigo debe poderse ejecutar varias veces sin fallar.**

In [22]:
#TODO - Escribe aquí tu codígo 
# Si necesitas ayuda revisa la documentación https://www.elastic.co/guide/en/elasticsearch/reference/7.10/indices-create-index.html


## Ejercicio 3. Trabajando con documentos.
En este apartado vamos a ver las operaciones básicas con documentos: creación, recuperación y borrado.

Aunque cuando estemos en produción siempre debemo crear el indice de manera explicita, ElaticSearch permite que insertemos documentos en un índice sin haberlo creado antes. El esque se inferirá de los datos introducidos.

In [28]:
request_body={
    "name":"Harry Potter",
    "birthday":"1980-07-31"
}
JSON(es.index(index="hp", id="harry", body=request_body))

<IPython.core.display.JSON object>

Ahora vamos a comprobar el mapping type del indice que hemos creado.

In [29]:
JSON(es.indices.get_mapping(index="hp"))

<IPython.core.display.JSON object>

Como podeis comprobar la inferencia ha detectado el tipo de los dos campos.

Ahora añadamos al resto de la pandilla.

In [30]:
request_body={
    "name":"Hermione Granger",
    "birthday":"1979-09-19"
}
JSON(es.index(index="hp",id="hermione", body=request_body))
request_body={
    "name":"Ron Weasley",
    "birthday":"1980-03-01"
}
JSON(es.index(index="hp",id="ron", body=request_body))
request_body={
    "name":"Ginny Weasley",
    "birthday":"1981-08-11"
}
JSON(es.index(index="hp",id="ginny",body=request_body))
request_body={
    "name":"George Weasley",
    "birthday":"1978-04-01"
}
JSON(es.index(index="hp",id="george", body=request_body))
request_body={
    "name":"Fred Weasley",
    "birthday":"1978-04-01"
}
JSON(es.index(index="hp", id="fred",body=request_body))

<IPython.core.display.JSON object>

Ahora vamos a recuperar a la pandilla.

In [33]:
JSON(es.search(index="hp"))

<IPython.core.display.JSON object>

Lamentablemente Fred ya no es parte de la pandilla, por lo que vamos a eliminarlo de la base de datos. Para ello buscamos su ID, en este caso el `5` y ejecutamos el siguiente comando.

In [34]:
JSON(es.delete(index="hp",id="fred"))

<IPython.core.display.JSON object>

Ahora borramos el indice.

In [35]:
JSON(es.indices.delete(index="hp",ignore=[400,404]))

<IPython.core.display.JSON object>

## Práctica
En esta sección vamos a repasar los comandos que hemos esta ejecutando en este ejercicio. Para ello vamos a desarrollar piezas de codigo que cumplan las indicaciones.

Lo primero que vamos a hacer es descargarnos el fichero csv con el que vamos a trabajar:


In [9]:
!wget https://gist.githubusercontent.com/aagea/8bbeef2112c969d70c41fa6f3882dadb/raw/25e5adc78b381ccc8f662dd5ee845a285395a13e/quotes.csv

--2024-01-30 14:13:20--  https://gist.githubusercontent.com/aagea/8bbeef2112c969d70c41fa6f3882dadb/raw/25e5adc78b381ccc8f662dd5ee845a285395a13e/quotes.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55360 (54K) [text/plain]
Saving to: ‘quotes.csv’

quotes.csv          100%[===================>]  54.06K  --.-KB/s    in 0.001s  

2024-01-30 14:13:21 (84.9 MB/s) - ‘quotes.csv’ saved [55360/55360]



Nuestro objetivo es indexar el contenido de es CSV en elasticsearch teniendo en cuenta lo siguiente:
- Los documento deben contener tres campos: quote, author y category.
- El campo category debe ser un array con los tres campos.
- Se debera generar un autonumerico que sera el id para cada uno de los documentos.

**Indexa todos los documento y haz la queries necesarias para demostrar que las operaciones funcionan correctamente**

In [9]:
#TODO - Escribe aquí tu codígo